# Video Capture and Frame Extraction

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edgeimpulse/utils-video-frame-extraction/blob/main/cam_video_capture_and_extract_frames.ipynb)

To use this script, run through the cells until you get to "Settings." Update the following settings:

* Edge Impulse API key
* Label

Run through the rest of the cells to capture a video, extract frames, and upload them to edge impulse with the label given by your "LABEL" setting. Repeat the whole process again for each class (change "LABEL").

In [ ]:
### Update Node.js to the latest stable version
!npm cache clean -f
!npm install -g n
!n stable

In [ ]:
### Install required packages and tools
!npm install -g --unsafe-perm edge-impulse-cli
!apt install ffmpeg

In [ ]:
### Use some packages
import random, os, shutil, uuid

In [ ]:
### Settings

# Label
LABEL = "sitting"

# Copy from Edge Impulse > your_project > Dashboard > Keys
EI_API_KEY = "ei_46c43c7abab0df0567d35edec8347440402d95c54d5b5013eeeddd5373b80033"

# ffmpeg framerate (e.g. "30" for 30 fps)
FRAME_RATE = "30"

# Working paths
BASE_PATH = "/content/"
TEMP_PATH = os.path.join(BASE_PATH, "temp")
OUTPUT_PATH = os.path.join(BASE_PATH, "output")

# Set aside 20% for test set (Edge Impulse automatically extracts validation set during training)
TEST_RATIO = 0.2

# Make it mostly random
random.seed()

## Video Capture Code

Credit goes to: https://androidkt.com/how-to-capture-and-play-video-in-google-colab/

In [ ]:
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64encode, b64decode

In [ ]:
def record_video(filename):
  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      
      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
    
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);

      video.srcObject = stream;
      video.muted = true;

      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);

      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
      video_file.write(binary)
    print(f"Finished recording video at:{filename}")
  except Exception as err:
    print(str(err))

def show_video(video_path, video_width = 600):
  
  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
### Record video
video_path = os.path.join(BASE_PATH, LABEL + ".mp4")
record_video(video_path)

In [ ]:
### Show video
show_video(video_path)

## Break video into frames

In [ ]:
### Delete and re-create working directories
if os.path.exists(TEMP_PATH):
  shutil.rmtree(TEMP_PATH)
os.makedirs(TEMP_PATH)
if os.path.exists(OUTPUT_PATH):
  shutil.rmtree(OUTPUT_PATH)
os.makedirs(OUTPUT_PATH)

In [ ]:
### Extract all frames from all videos and copy them to the appropriate directories
    
# Extract all frames in video into temp direcotry
print("Extracting:", LABEL + ".mp4")
!ffmpeg \
  -i {video_path} \
  -r {FRAME_RATE} \
  -hide_banner \
  -loglevel error \
  "{TEMP_PATH}/{LABEL}.%d.jpg"

# Append filenames with UUIDs and move to output directory
for filename in os.listdir(TEMP_PATH):
  file_path = os.path.join(TEMP_PATH, filename)
  id = str(uuid.uuid4().hex)[-12:]
  base_path = os.path.basename(file_path)
  file_stem = base_path.rsplit('.', 1)[0]
  uuid_file = file_stem + "_" + id + ".jpg"
  shutil.move(file_path, os.path.join(OUTPUT_PATH, uuid_file))

## Upload images to Edge Impulse

In [ ]:
### Send image files to Edge Impulse project

# Create list of file paths
paths = []
for filename in os.listdir(os.path.join(OUTPUT_PATH)):
  paths.append(os.path.join(OUTPUT_PATH, filename))

# Shuffle and divide into test and training sets
random.shuffle(paths)
num_test_samples = int(TEST_RATIO * len(paths))
test_paths = paths[:num_test_samples]
train_paths = paths[num_test_samples:]

# Create arugments list (as a string) for CLI call
test_paths = ['"' + s + '"' for s in test_paths]
test_paths = ' '.join(test_paths)
train_paths = ['"' + s + '"' for s in train_paths]
train_paths = ' '.join(train_paths)

# Use CLI tool to send training set to Edge Impulse
!edge-impulse-uploader \
--category training \
--label "{LABEL}" \
--api-key {EI_API_KEY} \
--silent \
--concurrency 20 \
{train_paths}

# Use CLI tool to send test set to Edge Impulse
!edge-impulse-uploader \
--category testing \
--label "{LABEL}" \
--api-key {EI_API_KEY} \
--silent \
--concurrency 20 \
{test_paths}